In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
locations = 'Lào Cai, Yên Bái, Điện Biên, Hòa Bình, Lai Châu, Sơn La, Hà Giang, Cao Bằng, Bắc Kạn, Lạng Sơn, Tuyên Quang, Thái Nguyên, Phú Thọ, Bắc Giang, Quảng Ninh, Bắc Ninh, Hà Nam, Hà Nội, Hải Dương, Hưng Yên, Hải Phòng, Nam Định, Ninh Bình, Thái Bình, Vĩnh Phúc, Thanh Hoá , Nghệ An'

In [4]:
import re
import sys

patterns = {
    '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
    '[đ]': 'd',
    '[èéẻẽẹêềếểễệ]': 'e',
    '[ìíỉĩị]': 'i',
    '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
    '[ùúủũụưừứửữự]': 'u',
    '[ỳýỷỹỵ]': 'y'
}

def convert(text):
    """
    Convert from 'Tieng Viet co dau' thanh 'Tieng Viet khong dau'
    text: input string to be converted
    Return: string converted
    """
    output = text
    for regex, replace in patterns.items():
        output = re.sub(regex, replace, output)
        # deal with upper case
        output = re.sub(regex.upper(), replace.upper(), output)
    return output

In [5]:
citys = convert(locations)
citys = [i.lower().strip() for i in citys.split(", ")]
citys=['/' + i.replace(" ",'-') for i in citys]
citys2ids = requests.get("https://thongtindoanhnghiep.co/api/city").json()['LtsItem']
citys2ids = [i for i in citys2ids if i['SolrID'] in citys]

In [6]:
# districts = []
# for city in citys2ids:
#     city_id = city['ID']
#     url = f'https://thongtindoanhnghiep.co/api/city/{city_id}/district'
#     r = requests.get(url)
#     districts += r.json()
    
# districts = [i['SolrID'][1:] for i in districts]

In [7]:
# num_records_by_districts = {}
# e = 0
# for i in districts:
#     e+=1
#     if(e%20) == 0:
#         print(e)
#     url = f'https://thongtindoanhnghiep.co/api/company?l={i}'
#     r = requests.get(url).json()
#     num_records_by_districts[i] = r["Option"]['TotalRow']
    

In [8]:
import pickle

# with open('num_records_by_districts.pickle', 'wb') as handle:
#     pickle.dump(num_records_by_districts, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [9]:

with open('num_records_by_districts.pickle', 'rb') as handle:
    num_records_by_districts = pickle.load(handle)



In [10]:
def get_closed_date(tax_code):
    url = f'https://thongtindoanhnghiep.co/api/company/{tax_code}'
    r = requests.get(url).json()
    closed_date = r['NgayDongMST']
    return closed_date


In [11]:
import random
import time
import pandas as pd

In [12]:
def parse_districts_html(district):
    num_records = num_records_by_districts[district]
    num_pages = num_records//15 + 2
    for page in range(1,num_pages):
        time.sleep(0.1)
        url = f'https://thongtindoanhnghiep.co/{district}?p={page}'
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        firms = soup.findAll("div", attrs={'class':'news-v3 bg-color-white'})
        res = [{'company_name': firm.find("h2").text, 
                  'location_detail': firm.findAll("strong")[-2].text,
                  'tax_code': firm.findAll("strong")[0].text,
                  'established_date': firm.findAll('p')[1].text.split(" ")[-1],
                  'sector': firm.findAll("strong")[-1].text,
                  'province': firm.findAll("strong")[1].text
                 } for firm in firms]
        df = pd.DataFrame(res, columns=['company_name','province','location_detail', 'sector', 'tax_code', 'established_date'])
        fn = district.split("/")[0] + '.csv'
        with open('./crawled/'+fn, 'a') as f:
            df.to_csv(f, index = False, header = False, sep='\t')
    print('DONE', district, num_records)

In [13]:
def parse_districts_api(district):
    num_records = num_records_by_districts[district]
    sleep = random.randint(1,10)
    time.sleep(sleep)
    num_pages = num_records//100 + 2
    for page in range(1,num_pages):
        time.sleep(0.1)
        url = f'https://thongtindoanhnghiep.co/api/company?r=100&p={page}&l={district}'
        try:
            r = requests.get(url).json()['LtsItems']
            res = [{'company_name': i['Title'], 
                      'location_detail': i['DiaChiCongTy'],
                      'tax_code': i['MaSoThue'],
                      'established_date': i['NgayCap'],
                      'sector': i['NganhNgheTitle'],
                      'province': i['TinhThanhTitle']
                     } for i in r]
            df = pd.DataFrame(res, columns=['company_name','province','location_detail', 'sector', 'tax_code', 'established_date'])
            fn = district.split("/")[0] + '.csv'
            with open('./crawled/' + fn, 'a') as f:
                df.to_csv(f, index = False, header = False, sep='\t')
        except:
            with open("./error.txt", 'a+') as f:
                print('error', url)

                time.sleep(120)
                f.write(f'{url}\n')
    print('DONE', district, num_records)

In [14]:
from multiprocessing import Pool
p = Pool(7)

error https://thongtindoanhnghiep.co/api/company?r=100&p=1&l=ha-noi/quan-dong-da
error https://thongtindoanhnghiep.co/api/company?r=100&p=7&l=ha-noi/quan-cau-giay


In [15]:
districts = num_records_by_districts.keys()

In [16]:
# p.map(parse_districts, [i for i in districts if num_records_by_districts[i] < 10000])

In [ ]:
remaining = [i for i in districts if i.startswith('ha-noi')]

In [ ]:
p.map(parse_districts_api, remaining)

In [ ]:
print('a')